In [ ]:
import sqlite3
import cv2
import numpy as np
import os


def cap_save(db_path, cascade_path):
    try:
        # Establish connection to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Load the Haar cascade classifier
        faceCascade = cv2.CascadeClassifier(cascade_path)

        # Initialize the webcam capture
        cap = cv2.VideoCapture(0)

        ret, frame = cap.read()

        if ret:
            # Convert to grayscale for face detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect faces using the cascade classifier
            faces = faceCascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30),
                flags=cv2.CASCADE_SCALE_IMAGE
            )

            if len(faces) > 0:
                # Select the first detected face
                (x, y, w, h) = faces[0]

                # Extract the region of interest (ROI)
                roi = gray[y:y+h, x:x+w]

                # Denoise the ROI
                denoised_roi = cv2.fastNlMeansDenoising(roi)

                # Convert the denoised ROI to byte array for database storage
                image_bytes = cv2.imencode('.jpg', denoised_roi)[1].tobytes()

                # Get a unique image ID (assuming an auto-incrementing ID in the table)
                cursor.execute("SELECT MAX(image_id) FROM images")
                max_id = cursor.fetchone()[0]
                if max_id is None:
                    image_id = 1
                else:
                    image_id = max_id + 1

                # Prompt user for name
                name = input("Enter name for the captured face: ")

                # Insert the image data and name into the database
                sql = "INSERT INTO images (image_id, image_name, image_data) VALUES (?, ?, ?)"
                cursor.execute(sql, (image_id, name, image_bytes))
                conn.commit()

                print("Face captured and saved successfully!")

                # Draw a rectangle around the detected face
                cv2.rectangle(gray, (x, y), (x+w, y+h), (255, 0, 0), 2)

                # Display the denoised ROI in a window
                cv2.imshow('Webcam', denoised_roi)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

              

            else:
                print("Error: No face detected.")

        else:
            print("Error: Failed to read from webcam.")

    except (cv2.Error, sqlite3.Error) as e:
        print(f"An error occurred: {e}")

    finally:
        # Close resources regardless of exceptions
        if cap.isOpened():
            cv2.waitKey(0)
            cap.release()
        cv2.destroyAllWindows()
        conn.close()

    return len(faces) > 0  # Return True if a face was captured


while True:
    db_path = input("Enter the path to the SQLite database file (e.g., C:/path/to/faces.db): ")
    if not os.path.exists(db_path):
        print(f"Error: Database file '{db_path}' does not exist. Please create it.")
    cascade_path = input("Enter the path to the Haar cascade classifier: ")
    if not os.path.exists(cascade_path):
        print(f"Error: cascade file '{cascade_path}' does not exist.")
    
   # db_path = 'C:\\Users\\asus\\Desktop\\test\\faces2.db'
    #cascade_path = "C:\\Users\\asus\\Desktop\\test\\haarcascade_frontalface_default.xml"
    success = cap_save(db_path, cascade_path)

    if success:
        repeat = input("Do you want to capture another face? (y/n): ")
        if repeat.lower() != 'y':
            break
    else:
        repeat = input("An error occurred. Do you want to try capturing another face? (y/n): ")
        if repeat.lower() != 'y':
            break
